[![Fixel Algorithms](https://fixelalgorithms.co/images/CCExt.png)](https://fixelalgorithms.gitlab.io)

# Optimization Methods

## Convex Optimization - Algorithms & Solvers - Alternating Direction Method of Multipliers (ADMM)

> Notebook by:
> - Royi Avital RoyiAvital@fixelalgorithms.com

## Revision History

| Version | Date       | User        |Content / Changes                                                   |
|---------|------------|-------------|--------------------------------------------------------------------|
| 1.0.000 | 04/10/2024 | Royi Avital | First version                                                      |

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/FixelAlgorithmsTeam/FixelCourses/blob/master/AIProgram/2024_02/0012LinearFitL1.ipynb)

In [ ]:
# Import Packages

# General Tools
import numpy as np
import scipy as sp
import pandas as pd

# Machine Learning

# Optimization
import cvxpy as cp
from sksparse.cholmod import cholesky

# Miscellaneous
import math
from platform import python_version
import random
import time

# Typing
from typing import Callable, List, Tuple, Union

# Visualization
from matplotlib.patches import Circle, Polygon
import matplotlib.pyplot as plt

# Jupyter
from IPython import get_ipython

## Notations

* <font color='red'>(**?**)</font> Question to answer interactively.
* <font color='blue'>(**!**)</font> Simple task to add code for the notebook.
* <font color='green'>(**@**)</font> Optional / Extra self practice.
* <font color='brown'>(**#**)</font> Note / Useful resource / Food for thought.

Code Notations:

```python
someVar    = 2; #<! Notation for a variable
vVector    = np.random.rand(4) #<! Notation for 1D array
mMatrix    = np.random.rand(4, 3) #<! Notation for 2D array
tTensor    = np.random.rand(4, 3, 2, 3) #<! Notation for nD array (Tensor)
tuTuple    = (1, 2, 3) #<! Notation for a tuple
lList      = [1, 2, 3] #<! Notation for a list
dDict      = {1: 3, 2: 2, 3: 1} #<! Notation for a dictionary
oObj       = MyClass() #<! Notation for an object
dfData     = pd.DataFrame() #<! Notation for a data frame
dsData     = pd.Series() #<! Notation for a series
hObj       = plt.Axes() #<! Notation for an object / handler / function handler
```

### Code Exercise

 - Single line fill

```python
vallToFill = ???
```

 - Multi Line to Fill (At least one)

```python
# You need to start writing
?????
```

 - Section to Fill

```python
#===========================Fill This===========================#
# 1. Explanation about what to do.
# !! Remarks to follow / take under consideration.
mX = ???

?????
#===============================================================#
```

In [ ]:
# Configuration
# %matplotlib inline

# warnings.filterwarnings("ignore")

seedNum = 512
np.random.seed(seedNum)
random.seed(seedNum)

# Matplotlib default color palette
lMatPltLibclr = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd', '#8c564b', '#e377c2', '#7f7f7f', '#bcbd22', '#17becf']
# sns.set_theme() #>! Apply SeaBorn theme
# sns.set_palette("tab10")

runInGoogleColab = 'google.colab' in str(get_ipython())

In [ ]:
# Constants

FIG_SIZE_DEF    = (8, 8)
ELM_SIZE_DEF    = 50
CLASS_COLOR     = ('b', 'r')
EDGE_COLOR      = 'k'
MARKER_SIZE_DEF = 10
LINE_WIDTH_DEF  = 2


In [ ]:
# Course Packages

from AuxFun import ProxGradientDescent
from AuxFun import DisplayCompaisonSummary, DisplayRunSummary, MakeSignal


In [ ]:
# Auxiliary Functions

def GenMatAVecBPolyhedron( mP: np.ndarray ) -> Tuple[np.ndarray, np.ndarray]:
    """
    Generates the matrix A and vector b that define the half space representation 
    of a 2D convex polyhedron given its vertex coordinates.

    The half space representation is of the form:
        A * x <= b
    where `A` is a matrix of normal vectors to the edges of the polyhedron and `b` 
    is a vector that defines the offset of each edge from the origin, ensuring that 
    the inequality is satisfied for all points inside the polyhedron.

    Parameters
    ----------
    mP : np.ndarray
        A `(numPts, 2)` array representing the coordinates of the three vertices of the 
        triangle, where each row corresponds to a vertex.  
        Type: `float` or `double`.

    Returns
    -------
    Tuple[np.ndarray, np.ndarray]
        mA : np.ndarray
            A `(numPts, 2)` array where each row is the normalized outward pointing normal 
            vector to one edge of the polygon.
            Type: `float` or `double`.
        vB : np.ndarray
            A `(n,)` array where each element is the dot product of the corresponding 
            normal vector with a point on the edge, defining the boundary of the 
            half space for each edge.
            Type: `float` or `double`.

    Notes
    -----
    - The edges are defined in a cyclic manner, such that edge 1 is between points 1 
      and 2, edge 2 is between points 2 and 3, and edge 3 is between points 3 and 1.
    - The normal vectors are computed to point towards the interior of the polyhedron 
      (Counter clockwise orientation). The are flipped later to hold the equation.
    - The function assumes that the vertices provided in `mP` are ordered in a 
      counter clockwise manner.
    """

    numPts = len(mP)

    # Initialize matrices A and b
    mA = np.zeros(shape = (numPts, 2)) #<! Matrix A for normal vectors
    vB = np.zeros(numPts)              #<! Vector b for boundary values

    # Loop over each edge of the polygon
    for ii in range(numPts):
        # Compute the direction vector of the current edge
        nextIdx = ((ii + 1) % numPts) #<! Ensure cyclic edges
        vE = mP[nextIdx] - mP[ii]     #<! Edge i: from Point i to Point i + i

        # Compute the normal vector (Perpendicular to the edge), inward pointing
        # The normal vector to a 2D vector (a, b) is (-b, a)
        vN = np.array([-vE[1], vE[0]])  #<! Normal to the edge
        
        # Normalize the normal vector
        vN = -vN / np.linalg.norm(vN) #<! Making it point outward by `-` to hold: `A x <= b`
        
        # Construct the matrix A (stack of normal vectors)
        mA[ii] = vN #<! Store the normal vector in matrix A

        # Compute the dot product of the normal vector with the current vertex
        vB[ii] = np.dot(vN, mP[ii])

    return mA, vB


def ProjectPolyhedron( vY: np.ndarray, mA: np.ndarray, vB: np.ndarray ) -> np.ndarray:
    """
    !!!The code is not verified!!!
    
    Project a point onto the polyhedron defined by the inequality Ax <= b using a closed form expression.
    
    This function implements the projection algorithm described in the paper "Closed Form Expressions 
    for Projectors onto Polyhedral Sets in Hilbert Spaces". The goal is to project a point 'vY' onto the polyhedron 
    defined by a set of linear inequality constraints given by 'mA' and 'vB'.
    
    !!!The code is not verified!!!
    
    Parameters:
    ----------
    vY : np.ndarray
        A vector representing the point to be projected onto the polyhedron.
        Shape: `(n,)`, where 'n' is the dimensionality of the point space.
        Type: `float` or `double`.
        
    mA : np.ndarray
        A matrix representing the linear inequality constraints. Each row of 'mA' corresponds 
        to a linear inequality of the form A_i x <= b_i.
        Shape: `(m, n)`, where 'm' is the number of constraints and 'n' is the dimensionality of the space.
        Type: `float` or `double`.
        
    vB : np.ndarray
        A vector representing the right-hand side of the inequality constraints Ax <= b.
        Shape: `(m,)`, where 'm' is the number of constraints.
        Type: `float` or `double`.
        
    Returns:
    -------
    vX : np.ndarray
        The projected point on the polyhedron. It is the closest point to 'vY' that satisfies the constraints.
        Shape: `(n,)`, where 'n' is the dimensionality of the point space.
        Type: `float` or `double`.
    
    Method:
    -------
    The algorithm works as follows:
    1. Compute the residual vector `r = A @ vY - b`, which indicates the violation of the constraints.
    2. Identify the set of "active constraints", i.e., the constraints that are violated (r_i > 0).
    3. Form a reduced system using only the active constraints and solve for the projection using 
       closed form expressions based on linear algebra.
    4. Compute the projection by adjusting the original point 'vY' based on the active constraints.
    
    This method is particularly useful for small to medium sized problems where a closed-form 
    projection is more efficient than iterative numerical methods like interior point or simplex solvers.
    
    Notes:
    ------
    - !!!The code is not verified!!!
    - This method assumes that the matrix formed by the active constraints (A_J A_J^T) is invertible.
    - For larger systems or cases where numerical stability is critical, using `np.linalg.solve` 
      (as in the implementation) is preferable to directly computing the matrix inverse.
    - In some edge cases, if the polyhedron is degenerate, special care might be needed for regularization.
    - !!!The code is not verified!!!
    
    """
    numRows = np.size(mA, 0)
    numCols = np.size(mA, 1)
    
    # Compute r = Ax - b
    vR = mA @ vY - vB #<! Residual
    
    # Determine the active set J
    vJ = np.where(vR > 0)[0]
    
    if len(vJ) == 0:
        return vY
    
    # Extract A_J and r_J
    mA_J = mA[vJ]
    vR_J = vR[vJ]
    
    # Compute the projection
    # vX = vY - mA_J.T @ AJAJt_inv @ vR_J
    vX = vY - mA_J.T @ np.linalg.solve(mA_J @ mA_J.T, vR_J)
    
    return vX

def ProjectHalfSpace( vY: np.ndarray, vA: np.ndarray, b: float ) -> np.ndarray:
    """
    Projects a point onto the boundary of a half space defined by a linear inequality.

    Given a point `vY`, this function checks if the point lies outside the half space 
    defined by the inequality `A * x <= b`. If the point is outside the half space, 
    it projects the point onto the boundary defined by the normal vector `A` and the 
    offset `b`. If the point is already within or on the boundary of the half space, 
    it remains unchanged.

    Parameters
    ----------
    vY : np.ndarray
        A 1D array representing the point to be projected.
        Type: `float` or `double`.

    vA : np.ndarray
        A 1D array representing the normal vector of the half-space.
        Type: `float` or `double`.

    b : float
        The scalar value defining the offset of the half-space.
        Type: `float`.

    Returns
    -------
    vX : np.ndarray
        A 1D array representing the projected point. If the input point is within or on 
        the boundary of the half-space, the output is the same as the input point. 
        If the point is outside the half-space, the output is the projected point on the 
        boundary.
        Type: `float` or `double`.

    Notes
    -----
    - The function assumes that `vA` is a non zero vector.
    - The projection is computed using the formula:
        vX = vY - (valR / np.inner(vA, vA)) * vA
      where `valR` is the distance from the point to the half space boundary.
    - The projection ensures that the resulting point satisfies the half space constraint.
    """

    valR = np.inner(vA, vY) - b

    if (valR > 0):
        vX = vY - ((valR / np.inner(vA, vA)) * vA)
    else:
        vX = vY
    
    return vX


In [ ]:
# Parameters

# Data

dataDim = 2

# Set of Unit Ball
circRadius  = 1
lCircCenter = [0.0, 0.0]

# Polyhedron
mP = np.array([[1.3,  0.2], [0.0, 1.2], [-0.85 * math.sqrt(2.0), -0.35 * math.sqrt(2.0)]])

# Point ot Project
vY = np.array([1.5, -1.5])

# Solver
ρ               = 2
numIterations   = 100

# # Verification
ε = 1e-6 #<! Error threshold

## Projection into Intersection of Convex Sets

The projection onto an intersection of sets is given by:

$$ \arg \min_{ \boldsymbol{x} } \frac{1}{2} {\left\| \boldsymbol{x} - \boldsymbol{y} \right\|}_{2}^{2} + \sum_{i}^{m} {\delta}_{\mathcal{C}_{i}} \left( \boldsymbol{x} \right) $$

Where ${\delta}_{\mathcal{C}_{i}} \left( \boldsymbol{x} \right)$  is the indicator function on whether $\boldsymbol{x} \in \mathcal{C}_{i}$.  

The naive solution, applying the projection onto each set iteratively:

$$ \boldsymbol{x} = \lim_{n \to \infty} {\left( \mathcal{P}_{\mathcal{C}_{1}} \circ \mathcal{P}_{\mathcal{C}_{2}} \circ \cdots \circ \mathcal{P}_{\mathcal{C}_{m}} \right)}^{n} \left( \boldsymbol{y} \right) $$

Converges to a point on the intersection of the sets. Yet it does not necessarily convergence to the _orthogonal projection_ onto the intersection of the sets.  
In case each set is a [Linear Sub Space](https://en.wikipedia.org/wiki/Linear_subspace) the method is guaranteed to converge.

The common method to solve the problem is the [Dykstra's Projection Algorithm](https://en.wikipedia.org/wiki/Dykstra%27s_projection_algorithm) which is equivalent to the ADMM with the _Consensus Trick_.

* <font color='brown'>(**#**)</font> The [Alternating Projection](https://en.wikipedia.org/wiki/Projections_onto_convex_sets) was analyzed by [John von Neumann](https://en.wikipedia.org/wiki/John_von_Neumann).

This notebooks covers:
 - Solving the _Projection into Intersection of Convex Sets_ problem using the ADMM with the _Consensus Trick_.  
 - Implementing the ADMM with _Consensus Trick_.

### The ADMM Method

* <font color='brown'>(**#**)</font> Deeper analysis is given in [StackExchange Mathematics - Orthogonal Projection onto the Intersection of Convex Sets](https://math.stackexchange.com/questions/1492095).



## Generate Data


The data is in 2D where there are 2 _convex sets_ and a starting point to project onto the sets.

In [ ]:
# Generate / Load the Data

oPatchCircle = Circle((0, 0), circRadius, alpha = 0.5, color = 'c', label = 'Circle')
oPatchPoly   = Polygon(mP, closed = True, alpha = 0.5, color = 'm', label = 'Polyhedron')

mA, vB = GenMatAVecBPolyhedron(mP)

# Analysis
mX = np.zeros(shape = (numIterations, dataDim)) #<! Initialization by zeros
mX[0] = vY #<! Using the initial point

dSolverData = {}


In [ ]:
# Display Data 

hF, hA = plt.subplots(figsize = (10, 6))
hA.plot(vY[0], vY[1], ls = 'None', marker = 'o', ms = 7, label = 'Starting Point')
hA.add_patch(oPatchCircle)
hA.add_patch(oPatchPoly)
hA.set_aspect('equal')
hA.set_xlim((-2, 2))
hA.set_ylim((-2, 2))

# Annotate the points of the Polyhedron
for ii, vP in enumerate(mP):
    annTxt = f'P{ii}'
    hA.annotate(annTxt, (vP[0], vP[1]), textcoords = "offset points", xytext =(0, 10), ha = 'center')

# Draw the lines of Half Spaces
vXX = np.linspace(-2, 2, 101)
for ii in range(mA.shape[0]):
    # The x, y formation of the line (Plotting friendly)
    # A[ii, 0] * x + A[ii, 1] * y = b[ii] -> y = (b[ii] - A[ii, 0] * x) / A[ii, 1]
    if mA[ii, 1] != 0: #<! Not vertical
        vYY = (vB[ii] - mA[ii, 0] * vXX) / mA[ii, 1]
        hA.plot(vXX, vYY, label = f'Line {ii + 1}: {mA[ii, 0]:0.2f} * ' + r'${x}_{1}$' + f' + {mA[ii, 1]:0.2f} * ' + r'${x}_{2}$' + f' = {vB[ii]:0.2f}')
    else:  # Vertical line (A[i, 1] == 0)
        vV = np.full_like(vXX, vB[ii] / mA[ii, 0])
        hA.plot(vV, vXX, label = f'Line {ii + 1}: x = {(vB[ii] / mA[ii, 0]):0.2f}')


hA.set_title('The Projection Problem')
hA.set_xlabel(r'${x}_{1}$')
hA.set_ylabel(r'${x}_{2}$')
hA.grid(True)
hA.legend();

* <font color='red'>(**?**)</font> What if the polygon was inside the circle?

## Projection onto an Intersection of Convex Sets

This section defines the problem and solve it using the _ADMM_.

### Objective Function

The objective function of the the projection:

$$ \arg \min_{\boldsymbol{x}} \frac{1}{2} {\left\| \boldsymbol{x} - \boldsymbol{y} \right\|}_{2}^{2} \; \text{ subject to } \boldsymbol{x} \in \bigcap_{i} \mathcal{C}_{i} $$

The projection onto a set is a common sub problem in many use cases:

 - **Image Restoration**: Projection the solution into the set of valid images.
 - **Spatial Localization**: Projection of the coordinates into a given polygon.
 - **Signal Processing**: Forcing some properties on the estimated signal.
 - **Machine Learning**: Forcing coefficients of a model to obey some rule set.

This requires a fast and effective solver of the problem.

* <font color='brown'>(**#**)</font> The problem can be solved using Quadratic Solver. Yet such solvers are not always available on edge devices.

In [ ]:
# Objective Function

#===========================Fill This===========================#
# 1. Implement the objective function. 
#    Given a vector of `vX` it returns the objective.
# 2. The implementation should be using a Lambda Function.
# !! You may `np.square()` and / or `np.linalg.norm()`.

hObjFun = lambda vX: 0.5 * np.square(np.linalg.norm(vX - vY))
#===============================================================#

* <font color='red'>(**?**)</font> How would the least squares (With no regularization, $\lambda = 0$) solution look like?

## Analysis

This section solves the problem in 2 ways:

 - DCP Solver: As the problem is _convex_ and relatively small it can be solved by a DCP solver.
 - ADMM: Using the _Consensus Trick_ to project onto the sets.


### DCP Solver

Solving the problem using a DCP Solver.

In [ ]:
# DCP Solution
# The Projection onto the Intersection of the Convex Sets.
# Solved using `CVXPY`.

startTime = time.time()

solverString = 'CVXPY'

#===========================Fill This===========================#
# 1. Create the auxiliary variable `vX`.
# 1. Define the objective function.
# 3. Define the constraints.
# 4. Solve the problem using `CVXPY`.
# !! You may use list operations to define constraints.

vX = cp.Variable(dataDim) #<! Objective Variable

cpObjFun = cp.Minimize(0.5 * cp.sum_squares(vX - vY)) #<! Objective Function
cpConst  = [mA @ vX <= vB, cp.norm(vX) <= circRadius] #<! Constraints
oCvxPrb  = cp.Problem(cpObjFun, cpConst) #<! Problem
#===============================================================#

oCvxPrb.solve(solver = cp.SCS)

assert (oCvxPrb.status == 'optimal'), 'The problem is not solved.'
print('Problem is solved.')

vX = vX.value

runTime = time.time() - startTime

In [ ]:
# Storing Results

DisplayRunSummary(solverString, hObjFun, vX, runTime, oCvxPrb.status)

dSolverData[solverString] = {'vX': vX, 'objVal': hObjFun(vX)}


In [ ]:
# Display Results

hA.plot(vX[0], vX[1], ls = 'None', marker = 'o', ms = 10, label = 'Reference Projection Point')
hA.legend()
display(hF)


### ADMM with _Consensus Trick_

This section implements the ADMM with _Consensus Trick_ Solver.  
It requires the projection (Prox) to each set.

#### Projection onto The ${L}_{2}$ Norm Unit Ball

The projection onto the set is given by:

$$ \arg \min_{ {\left\| \boldsymbol{x} \right\|}_{2}^{2} \leq 1} \frac{1}{2} {\left\| \boldsymbol{x} - \boldsymbol{y} \right\|}_{2}^{2} = \frac{1}{\max \left\{ {\left\| \boldsymbol{y} \right\|}_{2}, 1 \right\}} \boldsymbol{y} $$

* <font color='brown'>(**#**)</font> See [StackExchange Mathematics - Orthogonal Projection onto the ${L}_{2}$ Unit Ball](https://math.stackexchange.com/questions/627034).

In [ ]:
# Projection to the L2 Unit Ball

#===========================Fill This===========================#
# 1. Implement the function projecting onto the L2 Unit Ball. 
#    Given a vector `vY` it returns the projection at `vY`.
# 2. The implementation should be using a Lambda Function.
# !! The format should match Prox function (Parameter `λ`).
# !! You may find `np.reciprocal()` useful.

hProjL2BallFun = lambda vY, λ: np.reciprocal(max(np.linalg.norm(vY), 1)) * vY
#===============================================================#

#### Projection onto A Polyhedron

The interior of a convex [Polyhedron](https://en.wikipedia.org/wiki/Polyhedron) is defined by the set:

$$ \left\{ \boldsymbol{x} \mid \boldsymbol{A} \boldsymbol{x} \leq \boldsymbol{b} \right\}  $$

In 2D it creates a convex [Polygon](https://en.wikipedia.org/wiki/Polygon).

The set is intersection of [Half Spaces](https://en.wikipedia.org/wiki/Half-space_(geometry)).    
Hence the projection is done by projection to each half space, namely each row of $\boldsymbol{A}$ and the corresponding element in $\boldsymbol{b}$.  

The projection onto a half space is given by:

$$ \arg \min_{ \boldsymbol{a}^{T} \boldsymbol{x} \leq b } \frac{1}{2} {\left\| \boldsymbol{x} - \boldsymbol{y} \right\|}_{2}^{2} = \begin{cases}
\boldsymbol{y} & \text{ if } \; \boldsymbol{a}^{T} \boldsymbol{y} \leq b \\ 
\boldsymbol{y} - \frac{\boldsymbol{a}^{T} \boldsymbol{y} - b}{ {\left\| \boldsymbol{a} \right\|}_{2}^{2} } \boldsymbol{a} & \text{ if } \; \boldsymbol{a}^{T} \boldsymbol{y} > b
\end{cases} $$

* <font color='brown'>(**#**)</font> Half Spaces are not sub spaces.
* <font color='brown'>(**#**)</font> See [StackExchange Mathematics - Orthogonal Projection onto a Polyhedron (Matrix Inequality)](https://math.stackexchange.com/questions/4938099).
* <font color='brown'>(**#**)</font> See [StackExchange Mathematics - Orthogonal Projection onto a Half Space](https://math.stackexchange.com/questions/318740).
* <font color='brown'>(**#**)</font> There are papers which derives a closed form solution:
    - [Closed Form Expressions for Projectors onto Polyhedral Sets in Hilbert Spaces](https://arxiv.org/abs/1607.00102).
    - [Projecting onto Intersections of Half Spaces and Hyperplanes](https://arxiv.org/abs/2006.06995).
* <font color='brown'>(**#**)</font> Polyhedral are essential in [Linear Programming](https://en.wikipedia.org/wiki/Linear_programming). See [Polyhedral Geometry and Linear Optimization](https://www2.mathematik.tu-darmstadt.de/~paffenholz/daten/preprints/ln.pdf)

In [ ]:
# Projection to Half Space

#===========================Fill This===========================#
# 1. Implement the function projecting onto the a half space given `vA` and `b`. 
#    Given a vector `vY` it returns the projection at `vY`: `np.dot(vX, vA) <= b`.
# !! You may find https://math.stackexchange.com/questions/318740 useful.

# See `ProjectHalfSpace()`
#===============================================================#

In [ ]:
# Projectors List
# Create a List of Lambda Functions for Projection onto Half Space per row of `mA` and `hProjL2BallFun`.
# The first should be Prox of (rho / 2) * || x - y ||_{2}^{2} + (1 / 2) * || x - v ||.
# The format will match Prox functions.

lProxFun = [lambda vY, λ: ProjectHalfSpace(vY, mA[ii], vB[ii]) for ii in range(len(mA))]
lProxFun.append(hProjL2BallFun)
lProxFun.insert(0, lambda vV, λ: ((λ * vV) + vY) / (1 + λ))

### ADMM Solver

This section implements the ADMM with _Consensus Trick_.  

> [!TIP]  
> For $k \in \left\{ 1, 2, \ldots, K \right\}$ and for each $i$:
>    1. $\boldsymbol{x}{i}^{k} = \operatorname{prox}_{\frac{1}{\rho} {f}_{i}} \left( \boldsymbol{z}^{k - 1} - \boldsymbol{w}_{i}^{k - 1} \right)$
>    2. $\boldsymbol{z}^{k} = \frac{1}{m + 1} \sum_{i = 1}^{m + 1} \left( \boldsymbol{x}_{i}^{k} + \boldsymbol{w}_{i}^{k - 1} \right)$
>    3. $\boldsymbol{w}_{i}^{k} = \boldsymbol{w}_{i}^{k - 1} + \left( \boldsymbol{x}_{i}^{k} - \boldsymbol{z}^{k} \right)$


In [ ]:
# ADMM Function

#===========================Fill This===========================#
# 1. Implement the ADMM solver. 
#    Given a matrix `mX` with shape `(numIter, dataDim)` it applies the method.
# 2. An input parameter is `hMinFun` which is callable `hMinFun(vZ, vW, ρ)`.  
#    It minimizes the problem with respect to `vX`.
# 3. An input parameter is `hProxFun` which is callable `hProxFun(vY, λ)`.  
#    It minimizes the problem with respect to `vZ`.
# 4. The initial value is given by `mX[0, :]`.
# !! Do not overwrite `mX[0, :]`.
# !! Follow the doc string of the function.

def ADMMConsensus( mX: np.ndarray, lProxFun: List[Callable[[np.ndarray, float], np.ndarray]], /, *, ρ: float = 1.0 ) -> np.ndarray:
    """
    Alternating Direction Method of Multipliers (ADMM) for Consensus Optimization Problems.

    This function implements the ADMM algorithm to solve consensus optimization problems of the form:

        minimize f_1(x) + f_2(x) + ... + f_N(x)

    where each `f_i(x)` has an efficient proximal mapping. This type of problem arises when splitting a global optimization 
    problem across multiple independent subproblems, and each subproblem can be solved in parallel.

    The ADMM consensus formulation reformulates the problem as:

        minimize f_1(x_1) + f_2(x_2) + ... + f_N(x_N)
        subject to z = x_1 = x_2 = ... = x_N

    The idea is to decouple each subproblem but enforce agreement (consensus) across all variables via an auxiliary variable `z`. 
    The algorithm alternates between updating each local variable `x_i` using its proximal operator, updating the consensus variable `z`,
    and updating the dual variables.

    Parameters:
    ----------
    mX : np.ndarray
        A 2D array of shape `(numIter, dataDim)`, where each row represents a point in the optimization process.
        The initial point is provided in `mX[0]`, and subsequent points are updated in place.
        Type: `float` or `double`.

    lProxFun : List[Callable[[np.ndarray, float], np.ndarray]]
        A list of proximal operator functions, one for each subproblem `f_i(x)`. Each proximal operator takes two arguments:
        - The input point `vY`.
        - The penalty parameter λ.
          In this formulation it equals to `1 / ρ`.
        The proximal operator returns the solution to the local subproblem.

    ρ : float, optional
        Penalty parameter for the augmented Lagrangian. This parameter controls the tradeoff between the primal and dual residuals 
        in the optimization. Larger values of `ρ` put more emphasis on satisfying the constraint `P * x - z = 0`.
        Range: (0, inf).
        Default: `1.0`.

    Returns:
    -------
    mX : np.ndarray
        The updated sequence of points after performing the optimization. The final point can be found
        in `mX[-1]`.
    
    Notes:
    ------
    - ADMM consensus is useful in distributed or parallel optimization problems, where each `f_i(x)` corresponds to a subproblem that
      can be solved independently using its proximal operator.
    - The algorithm alternates between:
        1. **Updating each `x_i`**: Solve the local subproblem for each `f_i(x)` using the proximal operator.
        2. **Updating the consensus variable `z`**: Enforce agreement by averaging the `x_i` and dual variable `w_i`.
        3. **Updating the dual variables `w_i`**: Update the dual variable to enforce the constraint `x_i = z`.
    - The consensus variable `z` represents the shared variable across all subproblems, enforcing agreement between them.
    - ADMM is an efficient and flexible method for solving optimization problems that involve separable objectives and 
      linear constraints.
    - Convergence is typically achieved when the primal and dual residuals (measures of constraint violation and dual 
      update) become sufficiently small.
    """

    numIter = np.size(mX, 0)
    numSets = len(lProxFun)
    
    # The prox is usually given by: \arg \min_x 0.5 * || x - v ||^2 + paramLambda * g(x).
    # The ADMM form is: \arg \min_x (rho / 2) * || x - v ||^2 + g(x). 
    # Which means paramLamba = 1 / paramRho.
    ρInv = 1 / ρ #<! Basically paramLambda above.
    
    # Initialization
    mXX = np.tile(mX[0], (numSets, 1))
    vZZ = np.copy(mX[0]) #<! Separable variable
    mWW = np.zeros(shape = (numSets, dataDim)) #<! Dual variable (Lagrangian Multiplier)
    
    # Steady state
    for kk in range(1, numIter):
        for ii in range(numSets):
            mXX[ii] = lProxFun[ii](vZZ - mWW[ii], ρInv) #<! Solves \arg \min_x f_i(x) + (ρ / 2) * ||x - z + w ||_2^2
        vZZ[:]  = np.mean(mXX + mWW, axis = 0) #<! Solves \arg \min_z (ρ / 2) * \sum_i || x - z + w ||_2^2
        for ii in range(numSets):
            mWW[ii] += mXX[ii] - vZZ #<! Dual variable update
        
        mX[kk] = np.mean(mXX, axis = 0)
    
    return mX

#===============================================================#

In [ ]:
# ADMM Solution
# The Projection onto an Intersection of Convex Sets.
# Solved using ADMM with Consensus Trich Method.

startTime = time.time()

solverString = 'ADMM'

mX = ADMMConsensus(mX, lProxFun, ρ = ρ)

runTime = time.time() - startTime



In [ ]:
# Storing Results

vXX = np.copy(mX[-1])

DisplayRunSummary(solverString, hObjFun, vXX, runTime)

dSolverData[solverString] = {'vX': vXX, 'objVal': hObjFun(vXX), 'mX': np.copy(mX)}


### Display Results

In [ ]:
# Display Results

hF = DisplayCompaisonSummary(dSolverData, hObjFun)

* <font color='blue'>(**!**)</font> Change the Polyhedral parameters.

In [ ]:
# Display Data 

hA.plot(mX[:, 0], mX[:, 1], ls = 'None', marker = 'o', ms = 4, label = 'ADMM Solution Path')
hA.legend().remove()
hF = hA.get_figure()

display(hF)

* <font color='red'>(**?**)</font> What would happen if the intersection of the sets is empty?